In [2]:
import cv2
import os
import numpy as np
import tensorflow as tf
import time

from absl import app, flags, logging
from absl.flags import FLAGS

from models import RetinaFaceModel
from utils import (set_memory_growth, load_yaml, draw_bbox_landm,
                           pad_input_image, recover_pad_output)

ModuleNotFoundError: No module named 'RetinaFace'

In [1]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

logger = tf.get_logger()
logger.disabled = True
logger.setLevel(logging.FATAL)
set_memory_growth()

cfg = load_yaml('RetinaFace\\retinaface_res50.yaml')

# define network
model = RetinaFaceModel(cfg, training=False, iou_th=0.4, score_th=0.5)

# load checkpoint
checkpoint_dir = "\\" + cfg['sub_name']
checkpoint = tf.train.Checkpoint(model=model)
if tf.train.latest_checkpoint(checkpoint_dir):
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    print("[*] load ckpt from {}.".format(
        tf.train.latest_checkpoint(checkpoint_dir)))
else:
    print("[*] Cannot find ckpt from {}.".format(checkpoint_dir))
    exit()


NameError: name 'os' is not defined